In [7]:
from inpainting import inpainting
from besov_prior import besov_prior
from inverse_problem import inverse_problem
from rto_mh import rto_mh
import numpy as np

In [8]:
def Test_signal(x):
    Signal = np.zeros(len(x))
    interval_1 = np.logical_and(0.1 <= x, x <= 0.35)
    interval_2 = np.logical_and(0.35 < x, x <= 0.45)
    interval_3 = np.logical_and(0.45 < x, x <= 0.60)
    interval_4 = np.logical_and(0.60 < x, x < 0.90)
    Signal[interval_1] = np.exp(-(x[interval_1]-0.35)**2*150)
    Signal[interval_2] = 1.0
    Signal[interval_3] = 0.8
    Signal[interval_4] = 0.40
    return Signal

In [9]:
J = 9
n = 2**J
x = np.linspace(0, 1, 2**J, endpoint=False)
signal = Test_signal(x)

In [10]:
np.random.seed(5)
index_remove = np.concatenate((np.argwhere(np.logical_and(x>=0.2725,x<=0.3225)),np.argwhere(np.logical_and(x>=0.5725,x<=0.6225))),axis=0)[:,0]
index_remove_1 = np.argwhere(np.logical_and(x>=0.2725,x<=0.3225))[:,0]
index_remove_2 = np.argwhere(np.logical_and(x>=0.5725,x<=0.6225))[:,0]
index_remain = np.delete(np.linspace(0,len(x)-1,len(x),dtype=int),index_remove)
likelihood = inpainting(index_remove)
likelihood.set_data(signal,noise_level=2.0)
m = len(likelihood.data)
np.save("Inpainting_signal.npy",signal)
np.save("Inpainting_data.npy",likelihood.data)
np.save("Inpainting_region1.npy",index_remove_1)
np.save("Inpainting_region2.npy",index_remove_2)

In [11]:
s = 1.2
p = 1.5
wavelets = ['db1','db8']
delt = 0.025
level = 0
prior = besov_prior(J=J,delt=delt, level=level ,s=s, p=p,wavelet=wavelets[0])

In [ ]:
np.random.seed(5)
nsamp = 10000
x0 = 10*np.ones(len(x))
rto_sampler=rto_mh(x0,m+n,samp=nsamp)
for wavelet in wavelets:
    prior.wavelet = wavelet
    rto_sampler.x0 = np.ones(n)
    problem = inverse_problem(likelihood,prior,likelihood.jac_const(n) @ prior.jac_const(n))
    zMap=rto_sampler.initialize_Q(problem)
    rto_sampler.x0=10*zMap
    chain, acc_rate, index_accept, log_c_chain  = rto_sampler.sample(problem)
    print(acc_rate)
    np.save(wavelet + str(s) + str(p) + "Inpainting_samples.npy", chain)
    np.save(wavelet + str(s) + str(p) + "Inpainting_index_accept.npy",index_accept)
    rto_sampler.Q = np.identity(rto_sampler.Nrand)
    rto_sampler.eps = np.zeros(n+m)

In [13]:
s = [0.8, 1.4, 2.0]
p = [1.0, 1.5, 2.0]
wavelet = 'db8'
delt = 0.025
level = 0
prior = besov_prior(J=J,delt=delt, level=level ,s=s[0], p=p[0],wavelet=wavelet)


In [ ]:
np.random.seed(5)
nsamp = 10000
x0 = 10*np.ones(len(x))
rto_sampler=rto_mh(x0,m+n,samp=nsamp)
for i in range(len(p)):
    for j in range(len(s)):
        rto_sampler.x0 = np.ones(n)
        prior.p = p[i]
        prior.s = s[j]
        problem = inverse_problem(likelihood,prior,likelihood.jac_const(n) @ prior.jac_const(n))
        zMap=rto_sampler.initialize_Q(problem)
        xMap = prior.transform(zMap)
        rto_sampler.x0=10*zMap
        chain, acc_rate, index_accept, log_c_chain  = rto_sampler.sample(problem)
        print(acc_rate)
        np.save(wavelet + str(s[j]) + str(p[i]) + "Inpainting_samples.npy", chain)
        np.save(wavelet + str(s[j]) + str(p[i]) + "Inpainting_index_accept.npy",index_accept)
        rto_sampler.Q = np.identity(rto_sampler.Nrand)
        rto_sampler.eps = np.zeros(n+m)
